<img src="news.jpg">

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("train.csv")

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.describe()

,id,label
count,20800.000000,20800.000000
mean,10399.500000,0.500625
std,6004.587135,0.500012
min,0.000000,0.000000
25%,5199.750000,0.000000
50%,10399.500000,1.000000
75%,15599.250000,1.000000
max,20799.000000,1.000000


In [5]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [6]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
df=df.fillna('')

In [8]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [9]:
df.columns

Index(['id', 'title', 'author', 'text', 'label'], dtype='object')

In [10]:
df=df.drop(['id', 'title', 'author'], axis=1)

In [11]:
df.head()

,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,Ever get the feeling your life circles the rou...,0
2,"Why the Truth Might Get You Fired October 29, ...",1
3,Videos 15 Civilians Killed In Single US Airstr...,1
4,Print \nAn Iranian woman has been sentenced to...,1


In [12]:
from nltk.corpus import stopwords

In [13]:
from nltk.stem.porter import PorterStemmer

In [14]:
import re

In [15]:
port_stem=PorterStemmer()

In [16]:
port_stem

<PorterStemmer>

In [17]:
port_stem.stem("Hi thIs is Apurba * % %@@@")

'hi this is apurba * % %@@@'

In [18]:
def stemming(content):
    con=re.sub('[^a-zA-Z]', ' ', content)
    con=con.lower()
    con=con.split()
    con=[port_stem.stem(word) for word in con if not word in stopwords.words('english')]
    con=' '.join(con)
    return con

In [19]:
stemming('Hi this is apurba')

'hi apurba'

In [21]:
df['text']= df['text'].apply(stemming)

In [36]:
x=df['text']

In [37]:
y=df['label']

In [38]:
y.shape

(20800,)

In [39]:
from sklearn.model_selection import train_test_split

In [40]:
x_train , x_test , y_train, y_test = train_test_split(x, y, test_size=0.20)


In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [42]:
vect=TfidfVectorizer()


In [43]:
x_train=vect.fit_transform(x_train)
x_test=vect.transform(x_test)

In [44]:
x_test.shape

(4160, 98334)

In [45]:
from sklearn.tree import DecisionTreeClassifier

In [46]:
model=DecisionTreeClassifier()

In [47]:
model.fit(x_train, y_train)

DecisionTreeClassifier()

In [48]:
prediction=model.predict(x_test)

In [49]:
prediction

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

In [50]:
model.score(x_test, y_test)

0.8846153846153846

In [51]:
import pickle

In [52]:
pickle.dump(vect, open('vector.pkl', 'wb'))


In [53]:
pickle.dump(model, open('model.pkl', 'wb'))

In [54]:
vector_form=pickle.load(open('vector.pkl', 'rb'))

In [55]:
load_model=pickle.load(open('model.pkl', 'rb'))

In [57]:
def fake_news(news):
    news=stemming(news)
    input_data=[news]
    vector_form1=vector_form.transform(input_data)
    prediction = load_model.predict(vector_form1)
    return prediction

In [63]:
val=fake_news(""" project, nicknamed the â€˜Red-Med,â€™ was greenlit by Israel Prime Minister Benjamin Netanyahuâ€™s cabinet, and construction, which is expected to take five years, will begin within the year. About the link, Netanyahu said, â€œItâ€™s the first time weâ€™d be able to assist the countries in Europe and Asia to make sure they always have an open connection between Europe and Asia and between Asia and Europe.â€ While Israel is clearly poised to play a pivotal role in providing an unhindered and a secure trade route for European and Asia countries, Chinaâ€™s primary motivation is not to merely to build a network of allies in the Middle East. For China, the primary concern remains, apart from its trade with the Mid-Eastern countries, access to Europe and the African continent via the Middle East. However, both of these concerns cannot be effectively satisfied when the entire region remains under crisis. China, due to its fairly balanced approach to the crisis and the fact that it is perhaps the only global player that continues to enjoy equally good relations with such arch rivals as, for instance, Israel and Iran and Saudi Arabia and Iran, is an important country for the regional countries to look up to as a conflict mediator. This assumption about China playing this role owes its existence to the geography of trade China is eyeing to build through the Mid-East. As an alternative to shipping through the crowded Suez Canal, China sees value in sending goods into the Persian Gulf, and shipping them overland to the Mediterranean via several lines that could cross through Saudi Arabia, Kuwait, Qatar or even southern Iraq before finally continuing through Jordan and Israel. Will China eventually built this geography or not is a moot question. However, it is quite obvious that regional countries are open to it and know the potential benefits it can yield. Speaking at to the inaugural SIGNAL conference on Israelâ€™s China policy at the IDC in Herzliya in the last week of September, Capt. Yigal Maor, director general of the Israelâ€™s Transportation Ministryâ€™s Administration of Shipping and Ports, said that If China invests in building those paths, which he dubbed IGEC â€“ the Israel Gulf Economic Corridor â€“ it could push Gulf countries into more formal ties with Israel. â€œMaybe with the help of China there will be peace in that area,â€ he said. Whereas such prospects signal Israelâ€™s own elevation from the position of a US â€œprotectedâ€ state to an important power, integrating the region, as also continents, the potential entry of the â€œdragonâ€ in the region, connecting various countries into a whole, is likely to yield important results for the whole region. While we may have to wait a little to see what impact it leaves exactly, it is quite obvious that China is poised to become an important player in the Mideast security scene, and is already asserting its role economically, diplomatically and, to some extent, militarily. Salman Rafi Sheikh, research-analyst of International Relations and Pakistanâ€™s foreign and domestic affairs, exclusively for the online magazine â€œ New Eastern Outlook â€. Popular Articles""")

In [64]:
if val==[0]:
    print('reliable')
else:
    print('unreliable')

reliable
